This code snippet is intended to calculate the quantum yield of a photoluminescent sample by comparing its properties with another sample of known quantum yield, hence why we describe this method as quantum yield with standard. The protocol for the preparation and measurement of the samples needed for this method are detailed in (this document).